### Setup langfuse tracing

In [ ]:
from my_utils import setup_langfuse_tracer, hf_login
setup_langfuse_tracer()
hf_login()

### My LLM - To act as brain of all the below Agents

In [ ]:
from smolagents import LiteLLMModel
from my_config import MyConfig

my_config = MyConfig()

remote_url = F"http://{my_config.OLLAMA_INFERENCE_NODE_IP}:11434"
model = LiteLLMModel(
    base_url=remote_url
    , model_id='ollama/qwen2.5-coder:7b'
    # , model_id='ollama/llama3.2:latest'
)

### Start Building Agnents

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, VisitWebpageTool, FinalAnswerTool
from my_tools import suggest_menu, catering_service_tool, SuperheroPartyThemeTool
# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool(),
        FinalAnswerTool()
    ],
    model=model,
    max_steps=5,
    verbosity_level=2
)


In [ ]:
result = agent.run(
    "Can you give me the name of the highest-rated catering service in Gotham City?"
)
print(result)

In [ ]:
result = agent.run(
    "What would be a good superhero party idea for a 'villain masquerade' theme?"
)
print(result)

### Importing a Tool from the Hub

In [ ]:
from smolagents import load_tool, CodeAgent

image_generation_tool = load_tool(
    "m-ric/text-to-image",
    trust_remote_code=True
)

agent = CodeAgent(
    tools=[image_generation_tool],
    model=model
)

In [ ]:
agent.run("Generate an image of a luxurious superhero-themed party at Wayne Manor with made-up superheros.")

### Importing a Hugging Face Space as a Tool
You can also import a HF Space as a tool `using Tool.from_space()`. This opens up possibilities for integrating with thousands of spaces from the community for tasks from image generation to data analysis.

The tool will connect with the spaces Gradio backend using the gradio_client, so make sure to install it via pip if you don’t have it already.

For the party, Alfred can use an existing HF Space for the generation of the AI-generated image to be used in the announcement (instead of the pre-built tool we mentioned before). Let’s build it!


In [ ]:
from smolagents import CodeAgent, Tool

image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)
agent = CodeAgent(tools=[image_generation_tool], model=model)

In [ ]:
agent.run(
    "Improve this prompt, then generate an image of it.",
    additional_args={'user_prompt': 'A grand superhero-themed party at Wayne Manor, with Alfred overseeing a luxurious gala'}
)

### Importing a LangChain Tool
We’ll discuss the LangChain framework in upcoming sections. For now, we just note that we can reuse LangChain tools in your smolagents workflow!

You can easily load LangChain tools using the `Tool.from_langchain()` method. Alfred, ever the perfectionist, is preparing for a spectacular superhero night at Wayne Manor while the Waynes are away. To make sure every detail exceeds expectations, he taps into LangChain tools to find top-tier entertainment ideas.

By using `Tool.from_langchain()`, Alfred effortlessly adds advanced search functionalities to his smolagent, enabling him to discover exclusive party ideas and services with just a few commands.

Here’s how he does it:

In [ ]:
import os
from langchain.agents import load_tools
from smolagents import CodeAgent, Tool

os.environ['SERPAPI_API_KEY'] = my_config.SERPAPI_API_KEY
search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])
agent = CodeAgent(tools=[search_tool], model=model)

In [ ]:
agent.run("Search for luxury entertainment ideas for a superhero-themed event, such as live performances and interactive experiences.")